## Sources

In [17]:
import pandas as pd
import numpy as np
import csv
import uuid

### SKOS Concept

In [18]:
PATH = "/home/STual/KG-cadastre/data/SKOS_SOURCES.csv"

skos_srcs = pd.read_csv(PATH,header=0)
display(skos_srcs)

,skos:Concept,skos:inScheme,skos:broader,rdfs:label,skos:prefLabel@fr,skos:prefLabel@en,skos:altLabel@fr,skos:altLabel@en,skos:definition
0,Cadastre,CadastreSourceList,NaN,Cadastre,Cadastre,NaN,Cadastre napoléonien,NaN,Cadastre complet caractérisé par une numérotat...
1,Plan,CadastreSourceList,Cadastre,Plan,Plan,Map,NaN,NaN,Plan faisant partie de la documentation cadast...
2,TableauAssemblage,CadastreSourceList,Plan,Tableau d'assemblage,Tableau d'assemblage,Assembly map of the cadastral index maps,NaN,NaN,Plan réalisé à l'échelle d'une commune représe...
3,PlanParcellaire,CadastreSourceList,Plan,Plan parcellaire,Plan parcellaire,Cadastral index map,NaN,NaN,Plan représentant les parcellaires d'une porti...
4,PlanParcellaireMinute,CadastreSourceList,PlanParcellaire,Plan parcellaire : minute,Plan parcellaire : minute,Cadastral index map : working version,NaN,NaN,"Version de travail du plan parcellaire, peut c..."
5,PlanParcellaireFinal,CadastreSourceList,PlanParcellaire,Plan parcellaire final,Plan parcellaire final,Cadastral index map : final version,NaN,NaN,Version finale d'un plan parcellaire
6,PlanParcellaireIssuDAtlas,CadastreSourceList,PlanParcellaireFinal,Plan parcellaire issu d'atlas,Plan parcellaire issu d'atlas,Cadastral index map : final version extract fr...,NaN,NaN,Version finale d'un plan parcellaire extraire ...
7,RegistreCadastral,CadastreSourceList,Cadastre,Registre cadastral,Registre cadastral,Register,NaN,NaN,Registre faisant partie de la documentation ca...
8,MatriceCadastrale,CadastreSourceList,RegistreCadastral,Matrice cadastrale,Matrice cadastrale,Mutation register,NaN,NaN,Registre contenant l'ensemble des mutations af...
9,EtatsDeSections,CadastreSourceList,RegistreCadastral,Etats de sections,Etats de sections,Initial register,NaN,NaN,Registre d'état de sections décrivant le parce...


In [19]:
# Print the list of column headers
print(skos_srcs.columns.tolist())

['skos:Concept', 'skos:inScheme', 'skos:broader', 'rdfs:label', 'skos:prefLabel@fr', 'skos:prefLabel@en', 'skos:altLabel@fr', 'skos:altLabel@en', 'skos:definition']


In [20]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import SKOS, RDF, OWL

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/codes/cadastrenap/sourceType/")
g.bind('srctype', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
src = Namespace("http://data.ign.fr/def/sources#")
g.bind('cad', cad)
g.bind('src', src)

mlclasse= Namespace("http://data.ign.fr/id/codes/cadastrenap/mlClasse/")
g.bind('mlclasse', mlclasse)

# Iterate over each row in the DataFrame
for index, row in skos_srcs.iterrows():
    subject_uri = URIRef(baseuri + f"{str(row['skos:Concept'])}")
    g.add((subject_uri, RDF.type, OWL.NamedIndividual))
    g.add((subject_uri, RDF.type, cad.Source))
    g.add((subject_uri, RDF.type, SKOS.Concept))
    g.add((subject_uri, SKOS.inScheme, URIRef(cad + f"{str(row['skos:inScheme'])}")))
    g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:altLabel@fr']):
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:prefLabel@en']):
        g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@en"],lang='en')))

    if not pd.isnull(row['skos:altLabel@en']):
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@en"],lang='en')))

    if not pd.isnull(row['skos:broader']):
        g.add((subject_uri, SKOS.broader, URIRef(baseuri + f"{str(row['skos:broader'])}")))

    if not pd.isnull(row['skos:definition']):
        g.add((subject_uri, SKOS.definition, Literal(row["skos:definition"],lang='fr')))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastrenap/sourceType/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

srctype:ArticleDeClassement a cad:Source,
        owl:NamedIndividual,
        skos:Concept ;
    skos:altLabel "Numéro d'article"@fr ;
    skos:broader srctype:ZoneDePage ;
    skos:definition "Ligne de tableau de matrice qui décrit un état de parcelle"@fr ;
    skos:inScheme cad:CadastreSourceList ;
    skos:prefLabel "Ligne de matrice décrivant un état de parcelle"@fr .

srctype:ArticleDeMutation a cad:Source,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader srctype:ZoneDePage ;
    skos:definition "Zone ou cellule d'un folio ou sous-folio qui liste les propriétaires d'un ensemble de parcelles et indique leurs dates de mutations*"@fr ;
    skos:inScheme cad:CadastreSourceList ;
    skos:pre